In [166]:
in_colab = 'google.colab' in str(get_ipython())
if in_colab == True:
  !pip install yfinance
  import hvplot.pandas
  import plotly.express as px
  import yfinance as yf
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score
  import matplotlib.pyplot as plt
  from sklearn.kernel_ridge import KernelRidge
  import numpy as np
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
else:
  import hvplot.pandas
  import plotly.express as px
  import yfinance as yf
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score
  import matplotlib.pyplot as plt
  from sklearn.kernel_ridge import KernelRidge
  import numpy as np
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

In [167]:
crypto_btc = yf.download(tickers='BTC-USD', start='2017-12-01', end='2022-12-31').drop(columns=['Adj Close','High', 'Low']).round(2).reset_index()

[*********************100%***********************]  1 of 1 completed


In [168]:
crypto_btc

,Date,Open,Close,Volume
0,2017-12-01 00:00:00+00:00,10198.60,10975.60,6783119872
1,2017-12-02 00:00:00+00:00,10978.30,11074.60,5138500096
2,2017-12-03 00:00:00+00:00,11082.70,11323.20,6608309760
3,2017-12-04 00:00:00+00:00,11315.40,11657.20,6132409856
4,2017-12-05 00:00:00+00:00,11685.70,11916.70,6895260160
...,...,...,...,...
1851,2022-12-26 00:00:00+00:00,16842.25,16919.80,11886957804
1852,2022-12-27 00:00:00+00:00,16919.29,16717.17,15748580239
1853,2022-12-28 00:00:00+00:00,16716.40,16552.57,17005713920
1854,2022-12-29 00:00:00+00:00,16552.32,16642.34,14472237479


In [169]:
crypto_btc.reset_index(inplace=True)
fig = px.line(crypto_btc, x='Date', y='Close', title='Close Price with Range Slider')
fig.update_layout(hovermode="x")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=2, label="2y", step="year", stepmode="backward"),
            dict(step="all")
            
        ])
    )
)
fig.show()

In [170]:
crypto_btc2 = yf.download(tickers='BTC-USD', start='2023-01-01', end='2023-01-31').drop(columns=['Adj Close','High', 'Low']).round(2).reset_index()

[*********************100%***********************]  1 of 1 completed


In [171]:
'''
df = ()
def kernel_predict(df_pred):
    X = df.drop(columns=['Date'])
    y = df[['Close']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
    if df is crypto_btc:
        kr = KernelRidge(kernel='linear', alpha=5000)
    else:
        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)
    kr.fit(X_train, y_train)
    predict = kr.predict(X)
    if df is crypto_btc:
        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
    else:
        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})
    df_pred['Predicted Price'] = predict
    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100
    return df_pred
'''

"\ndf = ()\ndef kernel_predict(df_pred):\n    X = df.drop(columns=['Date'])\n    y = df[['Close']]\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)\n    if df is crypto_btc:\n        kr = KernelRidge(kernel='linear', alpha=5000)\n    else:\n        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)\n    kr.fit(X_train, y_train)\n    predict = kr.predict(X)\n    if df is crypto_btc:\n        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})\n    else:\n        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})\n    df_pred['Predicted Price'] = predict\n    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100\n    return df_pred\n"

In [172]:
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
crypto_btc_pred = crypto_btc.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
crypto_btc_pred['Predicted Price'] = predict
crypto_btc_pred['Price Accuracy']= (crypto_btc_pred['Predicted Price']/crypto_btc_pred['Actual Price']-1)*100
    

/Users/playb3yond40gb/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:196: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.



In [173]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

47.15996228448274
63.06413725831643
0.9999860064700025


In [174]:
crypto_btc_pred

,index,Date,Actual Price,Predicted Price,Price Accuracy
0,0,2017-12-01 00:00:00+00:00,10975.60,10980.0,0.040089
1,1,2017-12-02 00:00:00+00:00,11074.60,11084.0,0.084879
2,2,2017-12-03 00:00:00+00:00,11323.20,11334.0,0.095379
3,3,2017-12-04 00:00:00+00:00,11657.20,11651.0,-0.053186
4,4,2017-12-05 00:00:00+00:00,11916.70,11915.0,-0.014266
...,...,...,...,...,...
1851,1851,2022-12-26 00:00:00+00:00,16919.80,16948.0,0.166669
1852,1852,2022-12-27 00:00:00+00:00,16717.17,16766.0,0.292095
1853,1853,2022-12-28 00:00:00+00:00,16552.57,16612.0,0.359038
1854,1854,2022-12-29 00:00:00+00:00,16642.34,16680.0,0.226290


In [175]:
crypto_btc_pred['Price Accuracy'].describe()

count    1856.000000
mean        0.299234
std         0.286190
min        -0.698666
25%         0.129577
50%         0.228305
75%         0.389245
max         3.240464
Name: Price Accuracy, dtype: float64

In [184]:
crypto_btc_pred.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [176]:
print(f"BTC Kernel Ridge RMSE is {btc_rmse.round(2)}")

BTC Kernel Ridge RMSE is 63.06


In [198]:
future_df = pd.DataFrame()
  

future_df['Date'] = pd.date_range('2018-01-01', periods = 1856, freq ='1D')
future_df[['Close','Open','Volume']]= (0)

future_df

,Date,Close,Open,Volume
0,2018-01-01,0,0,0
1,2018-01-02,0,0,0
2,2018-01-03,0,0,0
3,2018-01-04,0,0,0
4,2018-01-05,0,0,0
...,...,...,...,...
1851,2023-01-26,0,0,0
1852,2023-01-27,0,0,0
1853,2023-01-28,0,0,0
1854,2023-01-29,0,0,0


In [200]:
X = crypto_btc.drop(columns=['Date'])
y = crypto_btc[['Close']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
kr = KernelRidge(kernel='linear', alpha=5000)
kr.fit(X_train, y_train)
predict = kr.predict(crypto_btc.drop(columns=['Date']))
btc_future = future_df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
btc_future['Predicted Price'] = predict

btc_future 

/Users/playb3yond40gb/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:196: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.



,Date,Actual Price,Predicted Price
0,2018-01-01,0,10980.0
1,2018-01-02,0,11084.0
2,2018-01-03,0,11334.0
3,2018-01-04,0,11651.0
4,2018-01-05,0,11915.0
...,...,...,...
1851,2023-01-26,0,16948.0
1852,2023-01-27,0,16766.0
1853,2023-01-28,0,16612.0
1854,2023-01-29,0,16680.0


In [201]:
btc_mae, btc_r_square, btc_r_square= (),(),()

btc_mae = mean_absolute_error(y['Close'], predict)
btc_rmse = np.sqrt(mean_squared_error(y['Close'], predict))
btc_r_square = r2_score(y['Close'], predict)
print(btc_mae)
print(btc_rmse)
print(btc_r_square)

47.15996228448274
63.06413725831643
0.9999860064700025


In [202]:
crypto_btc2 = yf.download(tickers='BTC-USD', start='2018-01-01', end='2023-01-30').drop(columns=['Adj Close','High', 'Low']).round(2).reset_index()
btc_future['Actual Price'] = crypto_btc2['Close'].round(0)
btc_future['Price Accuracy']= (btc_future['Predicted Price']/btc_future['Actual Price']-1)*100
btc_future.set_index('Date').tail(30)

[*********************100%***********************]  1 of 1 completed


,Actual Price,Predicted Price,Price Accuracy
Date,,,
2023-01-01,16625.0,17008.0,2.303759
2023-01-02,16688.0,17134.0,2.672579
2023-01-03,16680.0,16948.0,1.606715
2023-01-04,16863.0,17176.0,1.856135
2023-01-05,16837.0,17040.0,1.205678
2023-01-06,16952.0,17158.0,1.215196
2023-01-07,16955.0,16920.0,-0.206429
2023-01-08,17091.0,17288.0,1.152653
2023-01-09,17197.0,17174.0,-0.133744


In [206]:
btc_future.loc[btc_future['Date'] == '2020-01-27']

,Date,Actual Price,Predicted Price,Price Accuracy
756,2020-01-27,8910.0,7368.0,-17.306397


In [207]:
btc_future.loc[btc_future['Date'] == '2020-02-27']

,Date,Actual Price,Predicted Price,Price Accuracy
787,2020-02-27,8784.0,8916.0,1.502732


In [205]:
from bokeh.models.formatters import BasicTickFormatter

btc_future.hvplot(
    x='Date',
    y=[('Actual Price') ,'Predicted Price'],
    grid=True
)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)